In [5]:
from planet import api

client = api.ClientV1(api_key='98c3d498f425463ab3b3d3634bcef6b2')

# Load the shapefile
shapefile_path = 'c:/temp/planet/waterbody.shp'
shapefile_gdf = gpd.read_file(shapefile_path)
shapefile_wgs84  = shapefile_gdf.to_crs("epsg:4269")

# Get the bounding box coordinates
bounding_box = shapefile_wgs84.bounds.values[0]

print(bounding_box)

aoi = {
  "type": "Polygon",
  "coordinates": [bounding_box],
}

max_cloud_percentage = 0.1

# build a filter for the AOI
query = api.filters.and_filter(
  api.filters.geom_filter(aoi),
  api.filters.range_filter('cloud_cover', gt=0),
  api.filters.range_filter('cloud_cover', lt=max_cloud_percentage)
)

# we are requesting PlanetScope 4 Band imagery
item_types = ['PSScene4Band']
request = api.filters.build_search_request(query, item_types)

# this will cause an exception if there are any API related errors
results = client.quick_search(request)

# directory path to save images
save_dir = 'c:/temp/planet/download'
no_of_images = 1000

# items_iter returns an iterator over API response pages
for item in results.items_iter(no_of_images):
    # each item is a GeoJSON feature
    print(item['id'])
    assets = client.get_assets(item).get()
    activation = client.activate(assets['ortho_analytic_4b_sr'])

    callback = api.write_to_file(directory=save_dir)
    body = client.download(assets['ortho_analytic_4b_sr'], callback=callback)
    #body.await()


ImportError: cannot import name 'api' from 'planet' (C:\Anaconda\lib\site-packages\planet\__init__.py)